# Importación de librerías

In [2]:
import pandas as pd
import numpy as np
import os

# Funciones

In [13]:
def get_hours(x,y):
    val = y.replace(' ','')
    if val[-1]=='h':
        return val[:-1]
    elif val.isdigit():
        return val
    
    val = x.replace(' ','')
    if val[-1]=='h':
        return val[:-1]
    elif val.isdigit():
        return val
    else:
        return val

# Ruta de archivos

In [3]:
path = os.path.abspath(os.path.join(os.getcwd(), '..','2_data_understanding_module','collecting_initial_data','existing_data','DIRTY_DATA'))
path_additional_data = os.path.abspath(os.path.join(os.getcwd(), '..','2_data_understanding_module','collecting_initial_data','additional_data'))
path_etl = os.path.abspath(os.path.join(os.getcwd(),'CLEANED_DATA'))

# Costos de cursos

In [23]:
# Selecting Data
df_coec = pd.read_csv(os.path.join(path_additional_data,'costo_oferta_educacion_continua.csv'))
df_coec = df_coec[['Areas de Formación', 'Nombre  de los Programas','Modalidad', 'Tiempo de duración','Duración programa',' Precio ']]
df_coec = df_coec.rename(columns={' Precio ':'Precio'})
for col in df_coec.columns:
    df_coec[col] = df_coec[col].str.lower()
    aux = df_coec[col][0]
    for i in range(df_coec.shape[0]):
        if pd.isna(df_coec[col][i]):
            df_coec[col][i] = aux
        else:
            aux = df_coec[col][i]
            

df_coec = df_coec[(df_coec['Areas de Formación'].str.contains('tecnol|tics', regex=True))]
df_coec = df_coec[(df_coec['Precio'].str.contains('^(?:(?!llamar).)+$', regex=True))]

In [24]:
df_coec['carga_horaria'] = df_coec.apply(lambda x: get_hours(x['Tiempo de duración'], x['Duración programa']), axis=1)

# CAMBIO MANUAL DE DATOS
df_coec['carga_horaria'].iloc[-1] = 40

In [25]:
df_coec = df_coec.drop(['Areas de Formación','Tiempo de duración','Duración programa'],axis=1)

In [26]:
df_coec['Precio'] = df_coec['Precio'].str.replace('$','',regex=True)
df_coec['Precio'] = df_coec['Precio'].str.replace(',','',regex=True)
df_coec['Precio'] = df_coec['Precio'].str.replace(' ','',regex=True)
df_coec['Precio'] = df_coec['Precio'].astype('float64')

In [27]:
df_coec['carga_horaria'] = df_coec['carga_horaria'].astype('int32')
df_coec['precio'] = df_coec.pop('Precio')

In [28]:
df_coec['precio_por_hora'] = df_coec.apply(lambda x: np.round(x['precio']/x['carga_horaria'],decimals = 2), axis=1)

In [29]:
df_coec['Modalidad'] = df_coec['Modalidad'].replace({'en vivo':'online','virtual':'online'})

In [30]:
df_coec = df_coec.rename(columns={'Nombre  de los Programas':'curso','Modalidad':'modalidad'})

In [32]:
df_coec.to_csv(os.path.join(path_additional_data,'costos_cursos_educacion_continua_tec.csv'),index=False)

Trabajar manualmente los nombres de costos_cursos_educacion_continua_tec.csv

In [33]:
df_coec = pd.read_csv(os.path.join(path_additional_data,'costos_cursos_educacion_continua_tec_opt.csv'))


In [34]:
df_ccce = pd.read_csv(os.path.join(path_additional_data,'costos_de_cursos_cec_epn.csv'))
df_ccce = df_ccce.apply(lambda x: x.astype(str).str.lower())

In [37]:
df_costos_cursos = pd.concat([df_coec,df_ccce],axis=0)

In [38]:
df_costos_cursos['carga_horaria']=df_costos_cursos['carga_horaria'].astype('int64')
df_costos_cursos['precio']=df_costos_cursos['precio'].astype('float64')
df_costos_cursos['precio_por_hora']=df_costos_cursos['precio_por_hora'].astype('float64')

In [42]:
#df_costos_cursos.head()
#df_costos_cursos.info()

In [43]:
df_costos_cursos.to_csv(os.path.join(path_etl,'costos_estimados_cursos.csv'),index=False)

# Datos de Ubicación

In [45]:
df_ubicacion = pd.read_csv(os.path.join(path_additional_data,'Ubicacion.csv'))
df_ubicacion = df_ubicacion.apply(lambda x: x.astype(str).str.lower())
df_ubicacion = df_ubicacion.drop_duplicates()

In [9]:
#df_ubicacion.head()
#df_ubicacion.info()

In [46]:
df_ubicacion.to_csv(os.path.join(path_etl,'ubicacion.csv'),index=False)